In [ ]:
# A module to transform data to NumPy arrays and create random noise
import numpy as np 
from tensorflow.keras.models import Sequential
# Layers to be used in generator and discriminator
from tensorflow.keras.layers import Dense, LeakyReLU 
# A module to plot generated data 
import matplotlib.pyplot as plt 

In [ ]:
# Create variable X
X_train = np.linspace(-1, 1, 1000)
# Reshape X variable
X_train = X_train.reshape(-1, 1)
# Create variable Y such that y = x
y_train = X_train

In [ ]:
def build_generator(noise_dim):
  # Add layers one after one in the sequence
  model = Sequential() 
  # First fully connected layer
  model.add(Dense(16, activation = 'relu', kernel_initializer = 'he_uniform', input_dim = noise_dim))
  # Add LeakyReLU activation function
  model.add(LeakyReLU(alpha=0.01))
  # Output layer with the shape of 2
  model.add(Dense(2, activation = 'linear'))
  return model

In [ ]:
def build_discriminator():
  # Add layers one after one in the sequence
  model = Sequential() 
  # First fully connected layer
  model.add(Dense(32, activation = 'relu', kernel_initializer = 'he_uniform', input_dim = 2))
  # Add LeakyReLU activation function
  model.add(LeakyReLU(alpha=0.01))
  # Second fully connected layer
  model.add(Dense(8, activation = 'relu', kernel_initializer = 'he_uniform'))
  # Add LeakyReLU activation function
  model.add(LeakyReLU(alpha=0.01))
  # Output layer with the shape of 2
  model.add(Dense(1, activation = 'sigmoid'))
  return model

In [ ]:
def construct_models(noise_dim):
      # Build the discriminator model
      discriminator = build_discriminator() 
      # Compile the discriminator
      discriminator.compile(loss = 'binary_crossentropy' , optimizer = 'adam', metrics = ['accuracy'])
      # Build the generator model
      generator = build_generator(noise_dim) 
      # Freeze the discriminator during generator training
      discriminator.trainable = False
      gan = Sequential() 
      # Pit the generator and discriminator against each other
      gan.add(generator)
      gan.add(discriminator)
      # Compile the GAN model
      gan.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) 
      return generator, discriminator, gan

In [ ]:

# Define discriminator loss function
def disc_loss(model, X, y):
	      # Train the model on a batch of data and return loss value
	      return model.train_on_batch(X,y)
# Define generator loss function
def gen_loss(model, X, y):
	      # Train the model on a batch of data and return loss value
	      return model.train_on_batch(X,y)

In [ ]:

def print_generated_samples(batch_size, noise_dim):
      # Create test data
      X_test = np.random.uniform(low=-1, high =1, size=(batch_size//2, 1))
      y_test = X_test
      # Generate samples
      noise = np.random.normal(0, 1, (batch_size//2, noise_dim))
      gen_data = generator.predict(noise)  
      # Plot generated data
      plt.scatter(X_test, y_test, color = 'red')
      plt.scatter(gen_data[:,0], gen_data[:,1], color =  'blue')
      plt.xlim(-1,1)
      plt.ylim(-1,1)
      plt.legend(["real data", "generated data"], loc = "lower right")
      plt.show()

In [ ]:
def training(generator, discriminator, gan, noise_dim, epochs, batch_size):
  # Enumerate over epochs
  for e in range(epochs):
          # Random normal array for generator input 
          noise = np.random.normal(0, 1, (batch_size, noise_dim))
          # Create fake images by the generator
          fake_samples = generator.predict(noise)
          # Stack X and Y variable horizontally to build the dataset
          real_data = np.hstack((X_train, y_train))
          # Get a random real data points from the training data
          real_data = real_data[np.random.randint(0, real_data.shape[0], size = batch_size)]
          # Create real labels
          real_labels = np.ones((batch_size, 1))*0.9
          # Generate fake labels generated data points as zeros
          fake_labels = np.zeros((batch_size, 1))
          # Calculate the loss of real data points
          discriminator_loss_real = disc_loss(discriminator, real_data, real_labels)
          # Calculate the loss of generated data points
          discriminator_loss_fake = disc_loss(discriminator, fake_samples, fake_labels)
          # Compute total discriminator loss
          discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)
          # Generate random points as input for the generator
          x_gan = np.random.normal(0, 1, (batch_size, noise_dim))
          # generate real labels for gan
          y_gan = np.ones((batch_size, 1))
          #calculate the generator loss
          gan_loss = gen_loss(gan, x_gan, y_gan)
          # Print the progress
          if e % 10 == 0 or e == epochs-1:
            print('Epoch: ', e, ' Generator Loss: ', gan_loss , ' Discriminator Loss: ', discriminator_loss)
            print_generated_samples(batch_size, noise_dim)

In [ ]:
# Construct the models 
generator, discriminator, gan = construct_models(100) 
# Train GAN 
training(generator, discriminator, gan, 100, 10000, 128)